In [30]:
import pandas as pd
import iso8601
import pytz
from datetime import datetime, timedelta
from time import sleep

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, GetOrdersRequest, GetOrderByIdRequest
from alpaca.trading.enums import OrderSide, OrderStatus, TimeInForce
from alpaca.data.timeframe import TimeFrame
from alpaca.data.requests import StockLatestBarRequest, StockBarsRequest
from alpaca.data.historical import StockHistoricalDataClient

from bars import Bars

apikey = "PKKJZYB9P6Q36H84YMXF"
secretkey = "hBnEAEiD1f67p6hM4DKkUBtixY01YulNWSuGHOyx"

trading_client = TradingClient(apikey, secretkey, paper=True)


def get_clock():
    global trading_client
    clock = trading_client.get("/clock")
    return clock


def get_datetime_aware() -> datetime:
    clock = get_clock()
    timestamp = clock["timestamp"]
    dt = iso8601.parse_date(timestamp)
    return dt


def get_datetime_tz(timezone) -> datetime:
    dt = get_datetime_aware()
    return dt.astimezone(timezone)


def get_datetime_eastern() -> datetime:
    return get_datetime_tz(pytz.timezone("US/Eastern"))


def get_datetime_utc() -> datetime:
    return get_datetime_tz(pytz.timezone("UTC"))

def wait_for_market_open():
    while True:
        c = get_clock()
        if c["is_open"]:
            print("Market Open")
            return True
        print("is_open", c["is_open"])
        time_now = iso8601.parse_date(c["timestamp"])
        print("time_now", time_now)
        next_open = iso8601.parse_date(c["next_open"])
        print("next_open", next_open)
        seconds_to_open = (next_open - time_now).seconds
        print("seconds_to_open", seconds_to_open)
        if seconds_to_open < 3600:
            print("waiting for", seconds_to_open, "seconds")
            sleep(seconds_to_open)
        else:
            print(seconds_to_open, "seconds is too long to wait for open")
            return False


In [5]:
get_clock()

{'timestamp': '2022-09-26T21:20:16.853632689-04:00',
 'is_open': False,
 'next_open': '2022-09-27T09:30:00-04:00',
 'next_close': '2022-09-27T16:00:00-04:00'}

In [7]:
wait_for_market_open()

is_open False
time_now 2022-09-26 21:32:26.034552-04:00
next_open 2022-09-27 09:30:00-04:00
seconds_to_open 43053
43053 seconds is too long to wait for open


False

In [10]:
wait_for_market_open()
bars = Bars()

is_open False
time_now 2022-09-26 22:02:52.865375-04:00
next_open 2022-09-27 09:30:00-04:00
seconds_to_open 41227
41227 seconds is too long to wait for open


In [5]:
d = datetime(2022, 9,28)

In [8]:
d

datetime.datetime(2022, 9, 28, 0, 0)

In [22]:
ds = pytz.timezone("US/Eastern").localize(d.replace(hour=9,minute=30)).astimezone(pytz.utc)
de = ds + timedelta(hours=0.5)


2022-09-28 14:00:00+00:00


In [12]:
de

datetime.datetime(2022, 9, 28, 9, 30, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)

In [13]:
de.astimezone(pytz.utc)

datetime.datetime(2022, 9, 28, 13, 30, tzinfo=<UTC>)

In [28]:
d = datetime(2022, 9,28)
start = pytz.timezone("US/Eastern").localize(d.replace(hour=9,minute=30)).astimezone(pytz.utc)
end = start + timedelta(hours=0.5)
request_params = StockBarsRequest(
    symbol_or_symbols='SPY',
    timeframe=TimeFrame.Minute,
    start=start,
    end=end
)


In [31]:
stock_hist_client = StockHistoricalDataClient(apikey, secretkey)

In [33]:
bars = stock_hist_client.get_stock_bars(request_params)['SPY']

In [34]:
len(bars)

31

In [35]:
bars[0]

{   'close': 364.14,
    'high': 364.57,
    'low': 363.74,
    'open': 364.38,
    'symbol': 'SPY',
    'timestamp': datetime.datetime(2022, 9, 28, 13, 30, tzinfo=datetime.timezone.utc),
    'trade_count': 5192.0,
    'volume': 580871.0,
    'vwap': 364.140602}

In [36]:
bars[-1]

{   'close': 365.71,
    'high': 365.86,
    'low': 364.305,
    'open': 364.305,
    'symbol': 'SPY',
    'timestamp': datetime.datetime(2022, 9, 28, 14, 0, tzinfo=datetime.timezone.utc),
    'trade_count': 4430.0,
    'volume': 544918.0,
    'vwap': 365.236856}